In [1]:
import time 
from bs4 import BeautifulSoup
import requests
import pandas as pd
import random
import datetime
import csv
import undetected_chromedriver as uc
import pandas as pd
import cpca


In [3]:
df = pd.read_csv('page_entries_location.csv')

In [7]:
df.head()

,replyID,replyDate,reply_object,replyInfoHead,typeNameD,stateInfo,single_url,province,city,county
0,17193453,2023-01-23 10:19:00,山西省长治市长子县委书记,关于天然气停气,投诉/求助,待回复,http://liuyan.people.com.cn/threads/content?ti...,山西省,长治市,长子县
1,17167343,2023-01-16 19:26:00,山西省长治市上党区委书记,郝家庄镇岭上村供气不足供暖不足,投诉/求助,办理中,http://liuyan.people.com.cn/threads/content?ti...,山西省,长治市,NaN
2,17163718,2023-01-16 02:38:00,山西省太原市尖草坪区委书记,西留庄留苑小区暖气,投诉/求助,待回复,http://liuyan.people.com.cn/threads/content?ti...,山西省,太原市,尖草坪区
3,17038159,2022-12-28 05:10:00,山西省晋中市灵石县委书记,煤气公司每年腊月基本白天都在停气！,投诉/求助,已办理,http://liuyan.people.com.cn/threads/content?ti...,山西省,晋中市,灵石县
4,16999145,2022-12-23 08:37:00,山西省委书记蓝佛安,关于洪洞县堤村乡干河村天然气每晚停气,投诉/求助,待回复,http://liuyan.people.com.cn/threads/content?ti...,山西省,NaN,NaN


In [14]:
urls = df.single_url
urls

0       http://liuyan.people.com.cn/threads/content?ti...
1       http://liuyan.people.com.cn/threads/content?ti...
2       http://liuyan.people.com.cn/threads/content?ti...
3       http://liuyan.people.com.cn/threads/content?ti...
4       http://liuyan.people.com.cn/threads/content?ti...
                              ...                        
1051    http://liuyan.people.com.cn/threads/content?ti...
1052    http://liuyan.people.com.cn/threads/content?ti...
1053    http://liuyan.people.com.cn/threads/content?ti...
1054    http://liuyan.people.com.cn/threads/content?ti...
1055    http://liuyan.people.com.cn/threads/content?ti...
Name: single_url, Length: 1056, dtype: object

In [27]:
b = 500
for url in urls[500:]:
    b = b + 1
    driver = uc.Chrome()
    try:
        driver.get(url)
        content = BeautifulSoup(driver.page_source, 'html.parser')

        title = content.find('h1', class_='fl').text
        reply_object = content.find('div', class_='replyObject').text[5:]
        typeNameD = content.find('p', class_='typeNameD').text 
        domainName = content.find('p', class_='domainName').text
        stateInfo = content.find('p', class_='stateInfo').text

        #get the location info
        # df_location = cpca.transform(reply_object, pos_sensitive=False)


        replyInfoHead = content.find('h1', class_='fl').text
        replyName = content.find('li', class_='replyName').text
        replyMsg_org = content.find('li', class_='replyMsg').text

        replyID = replyMsg_org[3:11]
        replyDate = replyMsg_org[11:].replace(u'\xa0', u' ')
        replyContentMain = content.find('p', class_='replyContentMain').text
        #replace line break with space
        replyContentMain = replyContentMain.replace('\n', ' ')


        if stateInfo == '待回复':
            handleTime = None
            handleContent = None
        else:
            handleTime = content.find('div', class_='handleTime').text.replace(u'\xa0', u' ')
            handleContent = content.find('p', class_='handleContent').text
            handleContent = handleContent.replace('\n', ' ')


        single_reply = {}

        single_reply['replyID'] = replyID
        single_reply['replyDate'] = replyDate
        single_reply['typeNameD'] = typeNameD
        single_reply['title'] = title
        single_reply['reply_object'] = reply_object
        
        # single_reply['province'] = df_location['省']
        # single_reply['city'] = df_location['市']
        # single_reply['county'] = df_location['区']
        # single_reply['zipcode'] = df_location['adcode']

        # single_reply['replyInfoHead'] = replyInfoHead
        single_reply['domainName'] = domainName
        single_reply['stateInfo'] = stateInfo
        single_reply['replyName'] = replyName
        single_reply['replyContentMain'] = replyContentMain
        single_reply['handleTime'] = handleTime
        single_reply['handleContent'] = handleContent
        single_reply['url'] = url

        #save the reply
        with open('all_pages.csv', 'a', encoding='utf-8-sig', newline='') as f:
            writer = csv.writer(f)
            writer.writerow(single_reply.values())
        
        #close the window
        
        previewPic = content.find_all('div', class_='previewPic')
        if previewPic == None:
            print('no pic')
        else:
            a = 0
            for i in previewPic:
                a = a + 1
                #save the picture
                pic_url = i.find('img')['src']
                pic_name = replyID + '-' + str(a) + '.jpg'
                print(pic_url)
                print(pic_name)
                #using undetected_chromedriver to save the picture
                driver.get(pic_url)
                driver.save_screenshot(pic_name)
                time.sleep(random.randint(1,3))
        driver.close()
        print(b)
    except:
        print('error')
        driver.close()
        continue


501
502
503
504
505
506
507
http://liuyan.people.com.cn/files/attachment/month_2301/202301_X0tEeqhdwCXFfYBF8thnJsxbNWYXnNDm.jpg
17188030-1.jpg


/Users/VTL198/.pyenv/versions/3.11.0rc2/lib/python3.11/site-packages/selenium/webdriver/remote/webdriver.py:925: UserWarning: name used for saved screenshot does not match file type. It should end with a `.png` extension
  warnings.warn(


http://liuyan.people.com.cn/files/attachment/month_2301/202301_gzKPpm2RTgMBffRZHymx6sg8ZktCHe1j.jpg
17188030-2.jpg
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
error
552
553
554
555
556
557
558
559
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
http://liuyan.people.com.cn/files/attachment/month_2212/202212_WrX3kDlWn9TBX930D6NwM288RqEqrAOK.jpg
17058591-1.jpg
http://liuyan.people.com.cn/files/attachment/month_2212/202212_tQ21CXUzzI4NgCHntKmuEi971l07x8gf.jpg
17058591-2.jpg
http://liuyan.people.com.cn/files/attachment/month_2212/202212_Ri2YnUsgBmY9TnkVrO63wAXNply4WmVu.jpg
17058591-3.jpg
http://liuyan.people.com.cn/files/attachment/month_2212/202212_b32862DDBTfjJ2sxjpTqfZHQbM9NVVmB.jpg
17058591-4.jpg
http://liuyan.people.com.cn/files/attachment/month_2212/202212_pHnikTOn1gpI9stSrhGQptilvIpz2wPt

In [30]:
all_pages = pd.read_csv('all_pages.csv')

#adding a header to all_pages
all_pages.columns = ['replyID', 'replyDate', 'typeNameD', 'title', 'reply_object', 'domainName', 'stateInfo', 'replyName', 'replyContentMain', 'handleTime', 'handleContent', 'url']

In [33]:
all_pages

,replyID,replyDate,typeNameD,title,reply_object,domainName,stateInfo,replyName,replyContentMain,handleTime,handleContent,url
0,17167343,2023-01-16 19:26,投诉/求助,郝家庄镇岭上村供气不足供暖不足,山西省长治市上党区委书记,城建,办理中,吾***,自今年冬天入冬以来，上党区郝家庄镇岭上村天然气供气供暖都不足，每天一到做饭就没气，-15°的...,2023-01-17 19:31,您的留言我们已收到！您反映的问题已转相关部门处理，感谢您的宝贵留言！,http://liuyan.people.com.cn/threads/content?ti...
1,17163718,2023-01-16 02:38,投诉/求助,西留庄留苑小区暖气,山西省太原市尖草坪区委书记,城建,待回复,W***,尊敬的领导您好！我们这小区暖气每年晚来不说，来了家里面才13度，而且还是断断续续的给停。每次...,NaN,NaN,http://liuyan.people.com.cn/threads/content?ti...
2,17038159,2022-12-28 05:10,投诉/求助,煤气公司每年腊月基本白天都在停气！,山西省晋中市灵石县委书记,城建,已办理,m***,静升镇这儿的煤气公司每年腊月白天各种停气，也不发通知，也没有公告，打电话从来打不进去。蒸馒头...,2023-01-06 04:54,尊敬的人民网网友： 您好！您的留言已收悉，现将办理结果回复如下：相关单位已协调气源厂加压...,http://liuyan.people.com.cn/threads/content?ti...
3,16999145,2022-12-23 08:37,投诉/求助,关于洪洞县堤村乡干河村天然气每晚停气,山西省委书记蓝佛安,三农,待回复,8***,"我们村天然气每天晚上停气，严重影响到村民们正常生活和取暖问题,很多村民在村里反应，但是不给...",NaN,NaN,http://liuyan.people.com.cn/threads/content?ti...
4,16955074,2022-12-20 01:30,建言,取暖问题,山西省临汾市市长李云峰,三农,待回复,Z***,尊敬的领导你好 曲沃县裴南庄村三天两头停气，如此冷的天气，老人和...,NaN,NaN,http://liuyan.people.com.cn/threads/content?ti...
...,...,...,...,...,...,...,...,...,...,...,...,...
1045,16840497,2022-12-05 03:35,建言,农村天然气取暖,河北省委书记倪岳峰,三农,已办理,W***,领导您好：我是河北省邢台市隆尧县东良镇小干言村村民。现在已进入冬季，农村现在响应国家号召实行...,2023-01-19 03:49,网民您好！经协调督办，您反映的停气问题已得到解决。相关情况已与您进行了沟通。感谢您对我们工作...,http://liuyan.people.com.cn/threads/content?ti...
1046,16841053,2022-12-05 04:28,建言,我村经常停天然气,河北省委书记倪岳峰,三农,已办理,W***,由于天气很冷，我村天天停天然气，之前不让购买，现在让购买了天天停天然气，希望能尽快解决！！！,2023-01-19 03:48,网民您好！经协调督办，您反映的停气问题已得到解决。相关情况已与您进行了沟通。感谢您对我们工作...,http://liuyan.people.com.cn/threads/content?ti...
1047,16844344,2022-12-05 09:53,投诉/求助,鸡泽县中燃公司 限购天然气。经常停气，,河北省邯郸市市长樊成华,三农,已办理,可***,鸡泽县中燃公司 限购天然气。经常停气，老百姓无法正常过冬，又不让烧煤，寒心了,2022-12-27 02:54,网民您好！针对您反映的问题，市政府高度重视，责成鸡泽县政府核实办理，现就您反映的问题答复如下...,http://liuyan.people.com.cn/threads/content?ti...
1048,16845394,2022-12-05 17:23,投诉/求助,煤改气,河北省保定市委书记党晓龙,三农,已办理,马***,尊敬的书记好，我们是安国市南马村人，我们村煤改气前冬季取暖费3000以内就够了，改成天然气了...,2022-12-17 03:22,网民您好，您反映的问题我办已积极受理，交办至安国市发改局、住建局、南娄底乡调查解决。相关单位...,http://liuyan.people.com.cn/threads/content?ti...


In [34]:
df_location = cpca.transform(all_pages['reply_object'], pos_sensitive=False)

In [35]:
df_location 

,省,市,区,地址,adcode
0,山西省,长治市,None,上党区委书记,140400
1,山西省,太原市,尖草坪区,委书记,140108
2,山西省,晋中市,灵石县,委书记,140729
3,山西省,None,None,委书记蓝佛安,140000
4,山西省,临汾市,None,市长李云峰,141000
...,...,...,...,...,...
1045,河北省,None,None,委书记倪岳峰,130000
1046,河北省,None,None,委书记倪岳峰,130000
1047,河北省,邯郸市,None,市长樊成华,130400
1048,河北省,保定市,None,委书记党晓龙,130600


In [36]:
all_pages['province'] = df_location['省']
all_pages['city'] = df_location['市']
all_pages['county'] = df_location['区']
all_pages['zipcode'] = df_location['adcode']


In [37]:
all_pages_location = all_pages[['replyID', 'replyDate', 'typeNameD', 'title', 'reply_object', 'province', 'city', 'county', 'zipcode', 'domainName', 'stateInfo', 'replyName', 'replyContentMain', 'handleTime', 'handleContent', 'url']]

In [6]:
# read csv, ingore index
all_pages_location = pd.read_csv('all_pages_location.csv', index_col=0)

all_pages_location

,replyID,replyDate,typeNameD,title,reply_object,province,city,county,zipcode,domainName,stateInfo,replyName,replyContentMain,handleTime,handleContent,url
0,17167343,2023-01-16 19:26,投诉/求助,郝家庄镇岭上村供气不足供暖不足,山西省长治市上党区委书记,山西省,长治市,NaN,140400,城建,办理中,吾***,自今年冬天入冬以来，上党区郝家庄镇岭上村天然气供气供暖都不足，每天一到做饭就没气，-15°的...,2023-01-17 19:31,您的留言我们已收到！您反映的问题已转相关部门处理，感谢您的宝贵留言！,http://liuyan.people.com.cn/threads/content?ti...
1,17163718,2023-01-16 02:38,投诉/求助,西留庄留苑小区暖气,山西省太原市尖草坪区委书记,山西省,太原市,尖草坪区,140108,城建,待回复,W***,尊敬的领导您好！我们这小区暖气每年晚来不说，来了家里面才13度，而且还是断断续续的给停。每次...,NaN,NaN,http://liuyan.people.com.cn/threads/content?ti...
2,17038159,2022-12-28 05:10,投诉/求助,煤气公司每年腊月基本白天都在停气！,山西省晋中市灵石县委书记,山西省,晋中市,灵石县,140729,城建,已办理,m***,静升镇这儿的煤气公司每年腊月白天各种停气，也不发通知，也没有公告，打电话从来打不进去。蒸馒头...,2023-01-06 04:54,尊敬的人民网网友： 您好！您的留言已收悉，现将办理结果回复如下：相关单位已协调气源厂加压...,http://liuyan.people.com.cn/threads/content?ti...
3,16999145,2022-12-23 08:37,投诉/求助,关于洪洞县堤村乡干河村天然气每晚停气,山西省委书记蓝佛安,山西省,NaN,NaN,140000,三农,待回复,8***,"我们村天然气每天晚上停气，严重影响到村民们正常生活和取暖问题,很多村民在村里反应，但是不给...",NaN,NaN,http://liuyan.people.com.cn/threads/content?ti...
4,16955074,2022-12-20 01:30,建言,取暖问题,山西省临汾市市长李云峰,山西省,临汾市,NaN,141000,三农,待回复,Z***,尊敬的领导你好 曲沃县裴南庄村三天两头停气，如此冷的天气，老人和...,NaN,NaN,http://liuyan.people.com.cn/threads/content?ti...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,16840497,2022-12-05 03:35,建言,农村天然气取暖,河北省委书记倪岳峰,河北省,NaN,NaN,130000,三农,已办理,W***,领导您好：我是河北省邢台市隆尧县东良镇小干言村村民。现在已进入冬季，农村现在响应国家号召实行...,2023-01-19 03:49,网民您好！经协调督办，您反映的停气问题已得到解决。相关情况已与您进行了沟通。感谢您对我们工作...,http://liuyan.people.com.cn/threads/content?ti...
1046,16841053,2022-12-05 04:28,建言,我村经常停天然气,河北省委书记倪岳峰,河北省,NaN,NaN,130000,三农,已办理,W***,由于天气很冷，我村天天停天然气，之前不让购买，现在让购买了天天停天然气，希望能尽快解决！！！,2023-01-19 03:48,网民您好！经协调督办，您反映的停气问题已得到解决。相关情况已与您进行了沟通。感谢您对我们工作...,http://liuyan.people.com.cn/threads/content?ti...
1047,16844344,2022-12-05 09:53,投诉/求助,鸡泽县中燃公司 限购天然气。经常停气，,河北省邯郸市市长樊成华,河北省,邯郸市,NaN,130400,三农,已办理,可***,鸡泽县中燃公司 限购天然气。经常停气，老百姓无法正常过冬，又不让烧煤，寒心了,2022-12-27 02:54,网民您好！针对您反映的问题，市政府高度重视，责成鸡泽县政府核实办理，现就您反映的问题答复如下...,http://liuyan.people.com.cn/threads/content?ti...
1048,16845394,2022-12-05 17:23,投诉/求助,煤改气,河北省保定市委书记党晓龙,河北省,保定市,NaN,130600,三农,已办理,马***,尊敬的书记好，我们是安国市南马村人，我们村煤改气前冬季取暖费3000以内就够了，改成天然气了...,2022-12-17 03:22,网民您好，您反映的问题我办已积极受理，交办至安国市发改局、住建局、南娄底乡调查解决。相关单位...,http://liuyan.people.com.cn/threads/content?ti...


In [7]:
query = open('cnarea20200630.sql', 'r').read()

In [44]:
all_pages_location.to_csv('all_pages_location.csv', index=True)

In [58]:
df_zipcode = pd.read_csv('region_zipcode.csv')
df_zipcode.zipCode = df_zipcode.zipCode.astype(str)
df_zipcode.zipCode

0       100000
1       100010
2       100032
3       100000
4       100000
         ...  
3173       800
3174       900
3175       210
3176       950
3177       880
Name: zipCode, Length: 3178, dtype: object

In [59]:
#match the zipcode from df_zipcode to all_pages_location, and add the longitude and latitude to all_pages_location

for zip in all_pages_location.zipcode:
    if zip in df_zipcode.zipCode.values:
        print(zip)
        all_pages_location['longitude'] = df_zipcode[df_zipcode.zipCode == zip].longitude
        all_pages_location['latitude'] = df_zipcode[df_zipcode.zipCode == zip].latitude

152200
150500
150500
150400
150000
150100
150100
150000
150000
150000
150000
210000
210000
210000
210000
210000
210000
210000
210000
210000
210000
210000
210000
210000
211300
210000
210000
210000
210000
210000
210000
210000
210000
210000
230000
230000
230000
230000
230000
230000
230000
231100
230000
230000
230000
230000
230000
230000
230000
230000
231200
230000
230000
230000
230000
230000
230000
230000
230000
310000
330300
341600
410000
411400
410000
410000
410600
410600
410000
410000
410200
411400
411400
411400
410000
410000
421000
430100
450000
450000
450000
450000
450000
450000
510000
510000
510000
510000
511400
511300
511300
510000
512000
510000
510000
510000
510000
511600
510000
511500
511500
510000
511400
510000
512000
530600
530300
530600
530000
530100
530000
530000
530100
530600
530100
530000
530400
530300
610100
610400
610000
610000
610000
610000
610500
610100
610100
610100
610100
610100
610100
610100
610100
610100
610000
610100
610100
610100
610100
610100
610100
610100
610400

/var/folders/j_/_3yzbdjs3ylf4k27zrqrssqc0000gp/T/ipykernel_24287/3935948592.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_pages_location['longitude'] = df_zipcode[df_zipcode.zipCode == zip].longitude
/var/folders/j_/_3yzbdjs3ylf4k27zrqrssqc0000gp/T/ipykernel_24287/3935948592.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_pages_location['latitude'] = df_zipcode[df_zipcode.zipCode == zip].latitude
/var/folders/j_/_3yzbdjs3ylf4k27zrqrssqc0000gp/T/ipykernel_24287/3935948592.py:6: SettingWithC

130000
130400
130400
130400
130000
130600
130600
130600
130000
130600
130600
130000
130600
130000
130000
130300
130600
130400
130000
130600
130000
130400
130600
130000
130400
130400
130600
130600
130400
130000
130000
130600
130000
130600
130600
130000
130600
130000
130000
130400
130600
130600


In [60]:
all_pages_location

,replyID,replyDate,typeNameD,title,reply_object,province,city,county,zipcode,domainName,stateInfo,replyName,replyContentMain,handleTime,handleContent,url,longitude,latitude
0,17167343,2023-01-16 19:26,投诉/求助,郝家庄镇岭上村供气不足供暖不足,山西省长治市上党区委书记,山西省,长治市,None,140400,城建,办理中,吾***,自今年冬天入冬以来，上党区郝家庄镇岭上村天然气供气供暖都不足，每天一到做饭就没气，-15°的...,2023-01-17 19:31,您的留言我们已收到！您反映的问题已转相关部门处理，感谢您的宝贵留言！,http://liuyan.people.com.cn/threads/content?ti...,NaN,NaN
1,17163718,2023-01-16 02:38,投诉/求助,西留庄留苑小区暖气,山西省太原市尖草坪区委书记,山西省,太原市,尖草坪区,140108,城建,待回复,W***,尊敬的领导您好！我们这小区暖气每年晚来不说，来了家里面才13度，而且还是断断续续的给停。每次...,NaN,NaN,http://liuyan.people.com.cn/threads/content?ti...,NaN,NaN
2,17038159,2022-12-28 05:10,投诉/求助,煤气公司每年腊月基本白天都在停气！,山西省晋中市灵石县委书记,山西省,晋中市,灵石县,140729,城建,已办理,m***,静升镇这儿的煤气公司每年腊月白天各种停气，也不发通知，也没有公告，打电话从来打不进去。蒸馒头...,2023-01-06 04:54,尊敬的人民网网友： 您好！您的留言已收悉，现将办理结果回复如下：相关单位已协调气源厂加压...,http://liuyan.people.com.cn/threads/content?ti...,NaN,NaN
3,16999145,2022-12-23 08:37,投诉/求助,关于洪洞县堤村乡干河村天然气每晚停气,山西省委书记蓝佛安,山西省,None,None,140000,三农,待回复,8***,"我们村天然气每天晚上停气，严重影响到村民们正常生活和取暖问题,很多村民在村里反应，但是不给...",NaN,NaN,http://liuyan.people.com.cn/threads/content?ti...,NaN,NaN
4,16955074,2022-12-20 01:30,建言,取暖问题,山西省临汾市市长李云峰,山西省,临汾市,None,141000,三农,待回复,Z***,尊敬的领导你好 曲沃县裴南庄村三天两头停气，如此冷的天气，老人和...,NaN,NaN,http://liuyan.people.com.cn/threads/content?ti...,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1045,16840497,2022-12-05 03:35,建言,农村天然气取暖,河北省委书记倪岳峰,河北省,None,None,130000,三农,已办理,W***,领导您好：我是河北省邢台市隆尧县东良镇小干言村村民。现在已进入冬季，农村现在响应国家号召实行...,2023-01-19 03:49,网民您好！经协调督办，您反映的停气问题已得到解决。相关情况已与您进行了沟通。感谢您对我们工作...,http://liuyan.people.com.cn/threads/content?ti...,NaN,NaN
1046,16841053,2022-12-05 04:28,建言,我村经常停天然气,河北省委书记倪岳峰,河北省,None,None,130000,三农,已办理,W***,由于天气很冷，我村天天停天然气，之前不让购买，现在让购买了天天停天然气，希望能尽快解决！！！,2023-01-19 03:48,网民您好！经协调督办，您反映的停气问题已得到解决。相关情况已与您进行了沟通。感谢您对我们工作...,http://liuyan.people.com.cn/threads/content?ti...,NaN,NaN
1047,16844344,2022-12-05 09:53,投诉/求助,鸡泽县中燃公司 限购天然气。经常停气，,河北省邯郸市市长樊成华,河北省,邯郸市,None,130400,三农,已办理,可***,鸡泽县中燃公司 限购天然气。经常停气，老百姓无法正常过冬，又不让烧煤，寒心了,2022-12-27 02:54,网民您好！针对您反映的问题，市政府高度重视，责成鸡泽县政府核实办理，现就您反映的问题答复如下...,http://liuyan.people.com.cn/threads/content?ti...,NaN,NaN
1048,16845394,2022-12-05 17:23,投诉/求助,煤改气,河北省保定市委书记党晓龙,河北省,保定市,None,130600,三农,已办理,马***,尊敬的书记好，我们是安国市南马村人，我们村煤改气前冬季取暖费3000以内就够了，改成天然气了...,2022-12-17 03:22,网民您好，您反映的问题我办已积极受理，交办至安国市发改局、住建局、南娄底乡调查解决。相关单位...,http://liuyan.people.com.cn/threads/content?ti...,NaN,NaN
